In [ ]:
import pandas as pd

In [ ]:
df=pd.read_table("/content/drive/MyDrive/Rinex-AI/SpamHam.tsv")

In [ ]:
df

,label,message,length,punct
0,ham,"Go until jurong point, crazy.. Available only ...",111,9
1,ham,Ok lar... Joking wif u oni...,29,6
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155,6
3,ham,U dun say so early hor... U c already then say...,49,6
4,ham,"Nah I don't think he goes to usf, he lives aro...",61,2
...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,160,8
5568,ham,Will ü b going to esplanade fr home?,36,1
5569,ham,"Pity, * was in mood for that. So...any other s...",57,7
5570,ham,The guy did some bitching but I acted like i'd...,125,1


In [ ]:
df.nunique()

label         2
message    5169
length      274
punct        38
dtype: int64

In [ ]:
df.duplicated().sum()

403

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   label    5572 non-null   object
 1   message  5572 non-null   object
 2   length   5572 non-null   int64 
 3   punct    5572 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 174.2+ KB


In [ ]:
df['label'].value_counts()

ham     4825
spam     747
Name: label, dtype: int64

In [ ]:
df.isnull().sum()

label      0
message    0
length     0
punct      0
dtype: int64

In [ ]:
df.duplicated().sum()

403

In [ ]:
X=df['message'].values
y=df['label'].values

In [ ]:
X

array(['Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...',
       'Ok lar... Joking wif u oni...',
       "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's",
       ..., 'Pity, * was in mood for that. So...any other suggestions?',
       "The guy did some bitching but I acted like i'd be interested in buying something else next week and he gave it to us for free",
       'Rofl. Its true to its name'], dtype=object)

In [ ]:
y

array(['ham', 'ham', 'spam', ..., 'ham', 'ham', 'ham'], dtype=object)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=0)

In [ ]:
X_train.shape

(4179,)

In [ ]:
X_test.shape

(1393,)

#Understanding Vectorization

In [ ]:
s=['Congrats. You have won a lottery and you can get lottery money by calling lottery number','Give bank details for lottery','lottery for sure if bank details are verified']

In [ ]:
s[0].split()

['Congrats.',
 'You',
 'have',
 'won',
 'a',
 'lottery',
 'and',
 'you',
 'can',
 'get',
 'lottery',
 'money',
 'by',
 'calling',
 'lottery',
 'number']

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(stop_words='english')

In [ ]:
op=cv.fit_transform(s).toarray()
op

array([[0, 1, 1, 0, 3, 1, 1, 0, 0, 1],
       [1, 0, 0, 1, 1, 0, 0, 0, 0, 0],
       [1, 0, 0, 1, 1, 0, 0, 1, 1, 0]])

In [ ]:
cv.get_feature_names_out()

array(['bank', 'calling', 'congrats', 'details', 'lottery', 'money',
       'number', 'sure', 'verified', 'won'], dtype=object)

In [ ]:
data=pd.DataFrame(op,columns=cv.get_feature_names_out())
data

,bank,calling,congrats,details,lottery,money,number,sure,verified,won
0,0,1,1,0,3,1,1,0,0,1
1,1,0,0,1,1,0,0,0,0,0
2,1,0,0,1,1,0,0,1,1,0


#Continuing

In [ ]:
X_train_vect=cv.fit_transform(X_train)
X_test_vect=cv.transform(X_test)

In [ ]:
from sklearn.svm import SVC
model=SVC()
model.fit(X_train_vect,y_train)

SVC()

In [ ]:
y_pred=model.predict(X_test_vect)
y_pred

array(['ham', 'spam', 'ham', ..., 'ham', 'ham', 'ham'], dtype=object)

In [ ]:
y_test

array(['ham', 'spam', 'ham', ..., 'spam', 'ham', 'ham'], dtype=object)

In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix

In [ ]:
accuracy_score(y_test,y_pred)

0.9813352476669059

In [ ]:
confusion_matrix(y_test,y_pred)

array([[1208,    0],
       [  26,  159]])

In [ ]:
model.predict(cv.transform(["I am going"]))

array(['ham'], dtype=object)

In [ ]:
model.predict(cv.transform(['Give your bank details']))

array(['ham'], dtype=object)

Using a pipeline which is nothing but 

In [ ]:
from sklearn.pipeline import make_pipeline
text_model=make_pipeline(CountVectorizer(),SVC())
text_model

Pipeline(steps=[('countvectorizer', CountVectorizer()), ('svc', SVC())])

In [ ]:
text_model.fit(X_train,y_train)

Pipeline(steps=[('countvectorizer', CountVectorizer()), ('svc', SVC())])

In [ ]:
y_pred=text_model.predict(X_test)
y_pred

array(['ham', 'spam', 'ham', ..., 'ham', 'ham', 'ham'], dtype=object)

In [ ]:
import joblib
joblib.dump(text_model,'spam-ham')

['spam-ham']

In [ ]:
model=joblib.load('/content/spam-ham')
model.predict(['Congrats call the lottery number'])

array(['ham'], dtype=object)

In [ ]:
%%writefile app.py
import streamlit as st
import joblib
st.title("SPAM HAM CLASSIFIER")
text_model=joblib.load('/content/spam-ham')
ip=st.text_input("Enter your message: ")
op=text_model.predict([ip])
if st.button('Predict'):
  st.title(op[0])

Writing app.py


In [ ]:
!pip install streamlit --quiet

     |████████████████████████████████| 9.1 MB 7.0 MB/s 
     |████████████████████████████████| 235 kB 50.1 MB/s 
     |████████████████████████████████| 164 kB 38.9 MB/s 
     |████████████████████████████████| 78 kB 6.5 MB/s 
     |████████████████████████████████| 4.3 MB 41.4 MB/s 
     |████████████████████████████████| 181 kB 62.9 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
     |████████████████████████████████| 132 kB 60.2 MB/s 
     |████████████████████████████████| 132 kB 54.9 MB/s 
     |████████████████████████████████| 793 kB 46.0 MB/s 
     |████████████████████████████████| 423 kB 60.5 MB/s 
     |████████████████████████████████| 381 kB 69.9 MB/s 
     |████████████████████████████████| 51 kB 5.1 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
nbclient 0.6.6 requires traitlets>=5.2.2, but you have traitlets 5.1.1 which i

In [ ]:
!streamlit run app.py & npx localtunnel --port 8501

2022-07-21 13:55:13.612 INFO    numexpr.utils: NumExpr defaulting to 2 threads.

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://34.125.206.163:8501

npx: installed 22 in 5.716s
